In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import jieba

2021-10-13 21:43:46.035640: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
vocab_names = [
                'tag',
                'word',
              ]
vocabs = {}
for vocab_name in vocab_names:
  vocab_file =  f'../input/{vocab_name}_vocab.txt'
  if vocab_name != 'word':
    vocab = gezi.Vocab(vocab_file)
  else:
    vocab = gezi.Vocab(vocab_file, 200)
  vocabs[vocab_name] = vocab


In [3]:
vocabs['tag']

In [4]:
d = pd.read_csv('../input/info/infos.csv')

In [5]:
d.head(10)

,id,category_id,cat,subcat,tag_id,title,asr_text
0,8181862433864400302,10804,108,4,"55848798,800002360,593341,55744355,55859481,55...",我只不过跳了跳舞，结果光子实在太狠了,NaN
1,4155641279071878522,10401,104,1,"286226,101389,800000950,9504442,9820430",这么可爱的小姐姐，太搞怪了,邢飞真的是太能搞怪了，跟粉丝的速度一点都不装特别的，真就是喜欢这么正的女孩，你们是不是跟你一...
2,6281338907679991214,10501,105,1,"56080717,4093463,41614380,224444,101461,560840...",最近超级火的歌曲《冬眠》，美女唱得很不赖，绝对是实力派,你听。热车吗？你听着他说着你的。只上了赌注你。拍女生。啊。都要。的总结到位深。你替他掩饰着粽...
3,8506122561779211694,11604,116,4,"530228,3326413",下地干农活无意间拍下这一幕，这是什么东西，吓我一跳,一样。会落实。隆隆天空。给个反应。
4,4696077578276769146,10012,100,12,"101334,102376,800000171,10115859",和女儿们一块玩游戏，把她俩给骗了，她们却玩的很开心,你想。样子。认真在。让魔王魔王。是谁。
5,6290345740389858734,11604,116,4,"800000841,3326413,4356581,830719",猪肉太贵吃不起，撒网高手坐不住了，正在河沟撒网捕鱼呢！,提起吃肉我就上火。朱荣辉一个了不。赌。一斤猪肉。多百姓着，今有物伤多。鸡蛋，青菜和水果，随着...
6,8380021795080191406,10500,105,0,"5833128,3323093,55737387",沈老师跟我们告别回家休息了,那么吃饭点到底是？十二。怎么会。你不信。我现在会。现在却。切割。
7,3804367693936981422,11702,117,2,"102419,800001783,800000841,56048305,149957",这就是所谓的备胎吧,新闻我也。申请。肩膀的首先。SL。
8,2804562438649978286,11604,116,4,"1355431,165536,800001720,5865253",猪：你按呀，按破了喇叭，我也不走，哼哼,NaN
9,93400746928369018,12200,122,0,"527631,752678,438424,55803230",炎热的沙漠配上美女变得更火辣！,来家族。应该。你。到时。谁。


In [6]:
def gen_w2v(name, window=32, min_count=1, emb_dim=256):
  sentences = []
  for row in tqdm(d.itertuples(), total=len(d)):
    row = row._asdict()
    if name == 'word':
      l = ['[CLS]', *jieba.cut(str(row['title'])), '[SEP]', *jieba.cut(str(row['asr_text'])), '[SEP]']
    else:
      l = str(row[name]).split(',')
    sentences.append(l)
  ic(len(sentences))
  name = name.replace('_id', '')
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=mp.cpu_count(), epochs=10, callbacks=[monitor])
  ofile = f'../input/w2v/jieba/{emb_dim}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  vocab = vocabs[name]
  # emb = np.zeros([vocab.size(), emb_dim])
  emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
  for i in range(vocab.size()):
    word = vocab.key(i) 
    if word in w2v.wv:
      emb[i] = w2v.wv[word]
  ofile = f'../input/w2v/jieba/{emb_dim}/{name}.npy'
  np.save(ofile, emb)
  
  return w2v

In [8]:
# gen_w2v('tag_id', emb_dim=256)

  0%|          | 0/1138154 [00:00<?, ?it/s]

ic| 245958767.py:10 in gen_w2v()- len(sentences): 1138154


name: tag epoch: 1 model loss: 0.0 elapsed minutes: 0.13
name: tag epoch: 2 model loss: 0.0 elapsed minutes: 0.09
name: tag epoch: 3 model loss: 0.0 elapsed minutes: 0.08
name: tag epoch: 4 model loss: 0.0 elapsed minutes: 0.08
name: tag epoch: 5 model loss: 0.0 elapsed minutes: 0.09
name: tag epoch: 6 model loss: 0.0 elapsed minutes: 0.07
name: tag epoch: 7 model loss: 0.0 elapsed minutes: 0.08
name: tag epoch: 8 model loss: 0.0 elapsed minutes: 0.08


In [ ]:
# gen_w2v('tag_id', emb_dim=512)

In [ ]:
# gen_w2v('word', emb_dim=256)

In [ ]:
#  gen_w2v('word', emb_dim=512)

In [7]:
#  gen_w2v('word', emb_dim=768)

  0%|          | 0/1138154 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.661 seconds.
DEBUG:jieba:Loading model cost 0.661 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
ic| 737410769.py:10 in gen_w2v()- len(sentences): 1138154


name: word epoch: 1 model loss: 0.0 elapsed minutes: 30.47
name: word epoch: 2 model loss: 0.0 elapsed minutes: 28.90
name: word epoch: 3 model loss: 0.0 elapsed minutes: 32.69
name: word epoch: 4 model loss: 0.0 elapsed minutes: 31.31
name: word epoch: 5 model loss: 0.0 elapsed minutes: 31.30
name: word epoch: 6 model loss: 0.0 elapsed minutes: 30.35
name: word epoch: 7 model loss: 0.0 elapsed minutes: 29.46
name: word epoch: 8 model loss: 0.0 elapsed minutes: 29.23
name: word epoch: 9 model loss: 0.0 elapsed minutes: 34.51
name: word epoch: 10 model loss: 0.0 elapsed minutes: 34.38


In [8]:
gen_w2v('word', emb_dim=300)

  0%|          | 0/1138154 [00:00<?, ?it/s]

In [ ]:
gen_w2v('word', emb_dim=400)

In [ ]:
gen_w2v('word', emb_dim=600)